In [6]:
import os
import pandas as pd
from tqdm import tqdm

In [7]:
train_dir = "../data/raw/train"
val_dir = "../data/raw/val"
test_dir = "../data/raw/test"

In [20]:
def get_files(path_dir, ext='.parquet'):
    all_files = []
    for filename in os.listdir(path_dir):
        if filename.endswith(ext):
            file_path = os.path.join(path_dir, filename)
            all_files.append(file_path)
    
    return all_files

def save_audio(row, path_audio, path_label):
    audio = row['audio']['bytes']
    transcription = row['transcription']
    
    with open(path_audio, 'wb') as f:
        f.write(audio)
    
    with open(path_label, 'w', encoding='utf-8') as f:
        f.write(transcription)    
        
def process_data(all_files, root_dir, kind='train'):
    count = 0
    for f in all_files:
        print("Process file: ", f)
        data = pd.read_parquet(f)
        for i in tqdm(range(len(data))):
            row = data.iloc[i]
            path_audio = f"{root_dir}/audio/{kind}_{count}.wav"
            path_label = f"{root_dir}/label/{kind}_{count}.txt"
            
            save_audio(row, path_audio, path_label)
            count += 1

def remove_file(path_dir):
    for f in tqdm(os.listdir(path_dir)):
        if f.endswith('.wav'):
            os.remove(f"{path_dir}/{f}")
        elif f.endswith('.txt'):
            os.remove(f"{path_dir}/{f}")

def writeCSV(path_dir, kind='train'):
    data = []
    for name in os.listdir(f"{path_dir}/audio"):
        path_audio = f"{path_dir}/audio/{name}"
        path_label = f"{path_dir}/label/{name.split('.')[0]}.txt"
        
        with open(path_label, 'r', encoding='utf-8') as f:
            sentence = f.read()
        
        data.append([path_audio, sentence])
    
    df = pd.DataFrame(data=data, columns=['audio', 'sentence'])
    
    df.to_csv(f"{kind}.csv", index=False)

In [13]:
# Get file train, val, test
train_files = get_files(train_dir)
val_files = get_files(val_dir)
test_files = get_files(test_dir)

In [10]:
# Process data
process_data(train_files, train_dir, kind='train')
process_data(val_files, val_dir, kind='val')
process_data(test_files, test_dir, kind='test')

Process file:  ../raw/val/validation-00000-of-00002-b410ca42218b8815.parquet


100%|██████████| 3750/3750 [00:00<00:00, 4919.18it/s]


In [23]:
writeCSV("/home/j3s/NghiaND/ASR/data/raw/train", kind='train')
writeCSV("/home/j3s/NghiaND/ASR/data/raw/val", kind='val')
writeCSV("/home/j3s/NghiaND/ASR/data/raw/test", kind='test')